In [4]:
# This cell run FBA on a provided training and compute R2 between
# provided objective and calculated objective
# For exprimental training set medium input fluxes can be scaled by a value

from Library.Utilities import *
from Library.Build_Dataset import TrainingSet, run_cobra, get_minmed_varmed_ko
from sklearn.metrics import r2_score
DIRECTORY = './'

# What you can change 
seed = 10
np.random.seed(seed=seed)  
cobraname = 'iNF517_duplicated'  # name of the model 
mediumname = 'iNF517' # for running Cobra with RC predictions as inputs
method = 'EXP' # FBA, pFBA or EXP
# End of What you can change

# Get data
cobrafile =  DIRECTORY+'Dataset_input/'+cobraname
mediumfile = DIRECTORY+'Dataset_input/'+mediumname
parameter = TrainingSet(cobraname=cobrafile, 
                        mediumname=mediumfile, 
                        method=method,
                        verbose=False)
parameter.printout()
scaler_list = [1] # [0.95, 1, 1.05]
verbose = True # False

minmed, varmed, _ = get_minmed_varmed_ko(parameter.medium)
scaler = 1
valmed = {}

for m in minmed.keys():
    (l,v) = minmed[m]
    valmed[m] = v 
    
for j in range(len(list(varmed.keys()))):
    m = list(varmed.keys())[j]
    (l,v) = varmed[m]
    valmed[m] = v
    
out, y = run_cobra(parameter.model,
                   parameter.objective, 
                   valmed, method='FBA', verbose=False)
print('all', y)

med = varmed
for i in range(len(list(med.keys()))):
    m = list(med.keys())[i]
    valmed[m] = 0
    out, y = run_cobra(parameter.model,
                   parameter.objective, 
                   valmed, method='FBA', verbose=False)
    valmed[m] = v
    print(m, y)

"""
# regression cobra vs. true values
L = parameter.X.shape[0]
for scaler in scaler_list:
    Y = {}
    minmed, varmed, _ = get_minmed_varmed_ko(parameter.medium)
    for i in range(L):
        valmed = {}
        for m in minmed.keys():
            (l,v) = minmed[m]
            valmed[m] = v * scaler
        for j in range(len(list(varmed.keys()))):
            m = list(varmed.keys())[j]
            (l,v) = varmed[m]
            valmed[m] = v * parameter.X[i,j] * scaler
        try: 
            out, Y[i] = run_cobra(parameter.model, 
                                  parameter.objective, 
                                  valmed, 
                                  method='FBA', 
                                  verbose=False)
            if verbose:
                print(f'{i} {parameter.Y[i,0]} {Y[i]}')
        except:
            Y[i] = 0
            if verbose:
                print(f'{i} {parameter.Y[i,0]} {Y[i]} Cobra failed')
        
    Y = list(Y.values())
    r2 = r2_score(parameter.Y[0:L], Y[0:L], multioutput='variance_weighted')
    print(f'scaler {scaler:.3f} R2 {r2:.4f}')
"""

model file name: ./Dataset_input/iNF517_duplicated
reduced model: False
medium file name: ./Dataset_input/iNF517
medium: 115
experimental medium: 0
list of reactions in objective: ['BIOMASS_LLA']
method: EXP
trainingsize: 0
list of measured reactions: 1079
Stoichiometric matrix: (650, 1079)
Boundary matrix from reactions to medium: (28, 1079)
KO matrix from reactions to ko: (0, 1079)
Measurement matrix from reaction to measures: (1079, 1079)
Training set X: (0, 28)
Training set Y: (0, 0)
all 3.375596024911791
EX_glc__D_e_i 3.182826275491566
EX_xyl__D_e_i 3.375596024911795
EX_succ_e_i 3.375596024911795
EX_ala__L_e_i 3.375596024911795
EX_arg__L_e_i 3.352207805056756
EX_asn__L_e_i 3.355944743851111
EX_asp__L_e_i 3.3665298325327266
EX_cys__L_e_i 3.365313693576871
EX_glu__L_e_i 3.343467733341758
EX_gln__L_e_i 3.375596024911791
EX_gly_e_i 3.375596024911791
EX_his__L_e_i 3.350549613427875
EX_ile__L_e_i 3.3634436722423997
EX_leu__L_e_i 3.3755960249117916
EX_lys__L_e_i 3.346192243867105
EX_met_

"\n# regression cobra vs. true values\nL = parameter.X.shape[0]\nfor scaler in scaler_list:\n    Y = {}\n    minmed, varmed, _ = get_minmed_varmed_ko(parameter.medium)\n    for i in range(L):\n        valmed = {}\n        for m in minmed.keys():\n            (l,v) = minmed[m]\n            valmed[m] = v * scaler\n        for j in range(len(list(varmed.keys()))):\n            m = list(varmed.keys())[j]\n            (l,v) = varmed[m]\n            valmed[m] = v * parameter.X[i,j] * scaler\n        try: \n            out, Y[i] = run_cobra(parameter.model, \n                                  parameter.objective, \n                                  valmed, \n                                  method='FBA', \n                                  verbose=False)\n            if verbose:\n                print(f'{i} {parameter.Y[i,0]} {Y[i]}')\n        except:\n            Y[i] = 0\n            if verbose:\n                print(f'{i} {parameter.Y[i,0]} {Y[i]} Cobra failed')\n        \n    Y = list(Y

In [8]:
from Library.Utilities import *
from Library.Build_Dataset import TrainingSet, run_cobra, get_minmed_varmed_ko
from sklearn.metrics import r2_score
import cobra

def get_gene_id_from_name(cobramodel, gnames):
    # Return the gene IDs of the gene names
    gids = []
    for gname in gnames:
        for g in cobramodel.genes:
            if g.name == gname:
                gids.append(g.id)
    return gids


DIRECTORY = './Dataset_input/'

# What you can change 
name = 'iML1515EXP_duplicated'  # name of the model 

# List of specific gene IDs to check
gene_list = [
'eutA',
'eutB',
'eutC',
'glpQ',
'ugpQ',
'trpE',
'trpD',
'hisD',
'speG',
'speD',
'speE',
'patA',
'gadA',
'gadB',
'putA',
'proC',
'metE',
'metH',
'mmuM',
'avtA',
'betA',
'betB',
'glyA',
'kbl',
'serC',
'fadA',
'fadI',
'nanE',
'nanK',
'nadA',
'tyrA',
'tyrB',
'hyuA',
'dcd',
'nudG',
'cdh',
'argH',
'trpA',
'trpB',
'pyrF',
'codA',
'cdd',
'psuG',
'cysE',
'cysK',
'metB',
'speF',
'speB',
'speC',
'speA',
'adiA',
'alaA',
'alaC',
'gltB',
'gltD',
'gdhA',
'aspC',
'purA',
'purB',
'serB',
'dsdA',
'glnA',
'adeD',
'guaD',
'xapA',
'deoD',
'pheA',
'cmk',
'gltA',
'metC',
'malY',
'dcm',
'ltaE',
'thrBC',
'leuB',
'pncA',
'lysA',
'ilvED',
'pdxJ',
'pdxH',
'pdxY',
'pdxI',
'pdxK',
'ilvE',
'nanEK'
]  
gene_list = [gene.name for gene in model.genes]
gene_list = ['hisD', 'proC', 'serC', 'tyrA', 'trpA', 'pheA']


for gene_name in gene_list:
    gene_ids = get_gene_id_from_name(model, [gene_name])
    for gene_id in gene_ids:
        gene = model.genes.get_by_id(gene_id)
        for reaction in gene.reactions:
            print(f'{gene_name} {reaction.id}')


"""
for gene_id in gene_id_list:
    model = cobra.io.read_sbml_model(f'{DIRECTORY}{name}.xml')
    kos = get_gene_id_from_name(model, [gene_id])
    for ko in kos:
        model.genes.get_by_id(ko).knock_out()
    y = model.slim_optimize(error_value=0.0)
    print(f'{gene_id} {kos} {y:.3f}')
"""

hisD HISTD
proC P5CR
serC PSERT
serC SDPTA_for
serC OHPBAT_for
tyrA CHORM
tyrA PPND
trpA TRPS3
trpA TRPS2
trpA TRPS1
pheA CHORM
pheA PPNDH


"\nfor gene_id in gene_id_list:\n    model = cobra.io.read_sbml_model(f'{DIRECTORY}{name}.xml')\n    kos = get_gene_id_from_name(model, [gene_id])\n    for ko in kos:\n        model.genes.get_by_id(ko).knock_out()\n    y = model.slim_optimize(error_value=0.0)\n    print(f'{gene_id} {kos} {y:.3f}')\n"